# Pumped-Storage Optimisation with Genetic Algorithm and MILP

In [7]:
import pandas as pd
import datetime
import numpy as np
import plotnine as pn
import plotly.graph_objs as go
import plotly.express as px
from tqdm.notebook import tqdm
from IPython.display import clear_output, display
import os
from itertools import product

# Import own implementations
from milp import MILP
import genetic_numpy
from genetic_numpy import GA_discrete_actions, evaluate_fitness

# Importing tuning libraries
import ray
from ray import train, tune
from ray.tune.search.optuna import OptunaSearch
from ray.tune.schedulers import ASHAScheduler

background_colour = "#F2F2F2"
pn.theme_set(
    pn.theme_classic()
    + pn.theme(
        text=pn.element_text(family="monospace"),
        plot_background=pn.element_rect(
            fill=background_colour, colour=background_colour
        ),
        panel_background=pn.element_rect(
            fill=background_colour, colour=background_colour
        ),
        legend_background=pn.element_rect(
            fill=background_colour, colour=background_colour
        ),
    )
)

%load_ext blackcellmagic

The blackcellmagic extension is already loaded. To reload it, use:
  %reload_ext blackcellmagic


## Reading the Price data

In [8]:
df = pd.read_csv("../01 - Data/example_week.csv")
df.head(2)

,spot,utc_time
0,101.54,2022-01-01 00:00:00+00:00
1,52.13,2022-01-01 01:00:00+00:00


In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 168 entries, 0 to 167
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   spot      168 non-null    float64
 1   utc_time  168 non-null    object 
dtypes: float64(1), object(1)
memory usage: 2.8+ KB


## The Power Plant

In [10]:
plant_params = {
    "EFFICIENCY": 0.75,
    "MAX_STORAGE_M3": 5000,
    "MIN_STORAGE_M3": 0,
    "TURBINE_POWER_MW": 100,
    "PUMP_POWER_MW": 100,
    "TURBINE_RATE_M3H": 500,
    "MIN_STORAGE_MWH": 0,
    "INITIAL_WATER_LEVEL_PCT": 0,
}
plant_params["INITIAL_WATER_LEVEL"] = (
    plant_params["INITIAL_WATER_LEVEL_PCT"] * plant_params["MAX_STORAGE_M3"]
)
plant_params["PUMP_RATE_M3H"] = (
    plant_params["TURBINE_RATE_M3H"] * plant_params["EFFICIENCY"]
)
plant_params["MAX_STORAGE_MWH"] = (
    plant_params["MAX_STORAGE_M3"] / plant_params["TURBINE_RATE_M3H"]
) * plant_params["TURBINE_POWER_MW"]

## GA

In [11]:
ga_solver = GA_discrete_actions(
    plant_params=plant_params, spot=df["spot"], utc_time=df["utc_time"], actions_space= [-1, 0, 1]
)

In [12]:
analysis = ga_solver.tune(
    tune_config={
        "POP_SIZE": 500,
        "INITIAL_MUTATION_RATE": tune.uniform(0.05, 0.75),
        "FINAL_MUTATION_RATE": tune.uniform(0.001, 0.2),
        "MUTATION_ELEMENTS": tune.randint(1, df.shape[0]),
        "INITIAL_EXPLORATION": tune.uniform(0.05, 0.95),
        "ELITISM": tune.uniform(0.05, 0.95),
        "SURVIVAL_RATE": tune.uniform(0.05, 0.95),
    },
    total_generations=500,
    timeout_s=60*60*10,
)

2024-04-12 23:07:16,558	INFO worker.py:1743 -- Started a local Ray instance. View the dashboard at 127.0.0.1:8265 
2024-04-12 23:07:16,710	INFO packaging.py:530 -- Creating a file package for local directory '.'.
2024-04-12 23:07:16,715	WARNING packaging.py:405 -- File c:\Users\STEIM\OneDrive - BKW\Dokumente\ETH\genetic-algorithm-pumped-hydropower\05 - Part 1 Deterministic Actions\Actions Tuning.ipynb is very large (11.42MiB). Consider adding this file to the 'excludes' list to skip uploading it: `ray.init(..., runtime_env={'excludes': ['c:\Users\STEIM\OneDrive - BKW\Dokumente\ETH\genetic-algorithm-pumped-hydropower\05 - Part 1 Deterministic Actions\Actions Tuning.ipynb']})`
2024-04-12 23:07:16,747	WARNING packaging.py:405 -- File c:\Users\STEIM\OneDrive - BKW\Dokumente\ETH\genetic-algorithm-pumped-hydropower\05 - Part 1 Deterministic Actions\Tuning Results\202404111039_GA_Elite.csv is very large (34.73MiB). Consider adding this file to the 'excludes' list to skip uploading it: `ray.in

Trial name,fitness
train_00146635,318060
train_002a98f8,71882.5
train_003c4739,277504
train_0058867e,303701
train_005b7357,317230
train_00947315,316913
train_00c89730,319901
train_00ce1fbf,321235
train_00dab921,317034
train_00dc196f,321211


2024-04-12 23:09:41,626	WARNING util.py:202 -- The `callbacks.on_trial_result` operation took 0.896 s, which may be a performance bottleneck.
2024-04-12 23:09:41,629	WARNING util.py:202 -- The `process_trial_result` operation took 0.898 s, which may be a performance bottleneck.
2024-04-12 23:09:41,629	WARNING util.py:202 -- Processing trial results took 0.899 s, which may be a performance bottleneck. Please consider reporting results less frequently to Ray Tune.
2024-04-12 23:09:41,630	WARNING util.py:202 -- The `process_trial_result` operation took 0.900 s, which may be a performance bottleneck.
2024-04-12 23:34:10,131	WARNING util.py:202 -- The `callbacks.on_trial_result` operation took 0.943 s, which may be a performance bottleneck.
2024-04-12 23:34:10,133	WARNING util.py:202 -- The `process_trial_result` operation took 0.945 s, which may be a performance bottleneck.
2024-04-12 23:34:10,134	WARNING util.py:202 -- Processing trial results took 0.946 s, which may be a performance bott

In [13]:
analysis.best_config

{'POP_SIZE': 500,
 'INITIAL_MUTATION_RATE': 0.7107651734817916,
 'FINAL_MUTATION_RATE': 0.1969285330766747,
 'MUTATION_ELEMENTS': 116,
 'INITIAL_EXPLORATION': 0.8975767920426596,
 'ELITISM': 0.9088269091612435,
 'SURVIVAL_RATE': 0.3080815859946041}

In [14]:
tuning_history = (
    pd.concat(analysis.trial_dataframes, axis=0)
    .reset_index(drop=True)
    .filter(
        [
            "trial_id",
            "fitness",
            "training_iteration",
            "config/POP_SIZE",
            "config/INITIAL_MUTATION_RATE",
            "config/FINAL_MUTATION_RATE",
            "config/INITIAL_EXPLORATION",
            "config/ELITISM",
            "config/SURVIVAL_RATE",
        ]
    )
)

# Only keep the top combinations
num_best = 1000
top_combinations = tuning_history.groupby("trial_id").last().reset_index().sort_values("fitness", ascending=False).head(num_best)["trial_id"].to_list()
tuning_history = tuning_history.query("trial_id in @top_combinations")

In [15]:
(
    tuning_history
    .to_csv(
        f"./Tuning Results/{datetime.datetime.now().strftime('%Y%m%d%H%M')}_GA_Elite_numpy.csv",
        index=False,
    )
)